<a href="https://colab.research.google.com/github/frank-895/train_learner/blob/main/learner_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

MNIST contains images of handwritten digits. We will use these images to manually create a Learner model that can detect the difference between handwritten numbers.

In [ ]:
path = untar_data(URLs.MNIST)
Path.BASE_PATH = path
(path).ls()

(#2) [Path('training'),Path('testing')]

We will download the training and validation images for each number into respective arrays, converting them to a scale of 0 to 1.

In [ ]:
train_tns = []
valid_tns = []
# extract and convert to scale of 0 to 1 using tensor operations
for i in range(10):
  train_tns.append(torch.stack([tensor(Image.open(o)).float()/255 for o in (path/'training'/str(i)).ls()]))
  valid_tns.append(torch.stack([tensor(Image.open(o)).float()/255 for o in (path/'testing'/str(i)).ls()]))

Now, we need to convert our tensors from a list of matrices to a list of vectors, flattening the pixels in each image to a single row of 28*28 pixels. We use `view` which increases the axis to make it as big as needed to fit all the data. Then, we will create labels for each of the images.

In [ ]:
train_x = torch.cat(train_tns).view(-1, 28*28)
train_y = []
for i in range(10):
  train_y.extend([i] * len(train_tns[i]))
train_y = tensor(train_y).unsqueeze(1)

In [ ]:
print(train_y)

tensor([[0],
        [0],
        [0],
        ...,
        [9],
        [9],
        [9]])
